# Rho $\gamma_4$-current

In [ ]:
### Parameter list ###

# Commonly changed parameters
g = 8
p_src = (0,-1,0)
p_snk = (0,0,0)
make_hermitian = false
fit_data = false
hide_prog = false

# Other parameters
size_s = 16
size_t = 60
m = "-8999"
q = (p_snk[1]-p_src[1],p_snk[2]-p_src[2],p_snk[3]-p_src[3])
sources = ["DG0_1", "DG1_1", "DG1_1", "DG2_1", "DG2_1"]
sinks = ["DG0_1", "DG1_1", "DG1_1", "DG2_1", "DG2_1"]
t_gen_ev = 0
t_var = 10
t_sn = 10
t_sink = [15, 18, 21]
g_ins = 1
had = "rho_x"
seqsource = "a0-rho_x_1"
file_prefix = ["/home/arios/Documents/LQCDConfigs/wil_16_60_aniso_cluster/rho/t$(t_sink[i])/" for i in 1:length(t_sink)]

if g in [1,2,4,8,7,11,13,14]
    curr = "nonlocal"
else
    curr = "local"
end;

In [ ]:
### Load modules and data ###

push!(LOAD_PATH, pwd())
using lqcd, lqcdfits, PyPlot, Interact

had_3ptfn = [read_bar3ptfn_file(seqsource, curr, g_ins, g, q, p_snk, sources, sinks, file_prefix[i]) for i in 1:length(t_sink)]
had_2ptfn_src = [read_hadspec_file(had, m, p_src, sources, sinks, file_prefix[i]) for i in 1:length(t_sink)]
had_2ptfn_snk = [read_hadspec_file(had, m, p_snk, sources, sinks, file_prefix[i]) for i in 1:length(t_sink)];

if make_hermitian
    [make_hermitian!(had_3ptfn[i]) for i in 1:length(t_sink)]
    [make_hermitian!(had_2ptfn_src[i]) for i in 1:length(t_sink)]
    [make_hermitian!(had_2ptfn_snk[i]) for i in 1:length(t_sink)]
end;

In [ ]:
### Process data ###

had_mass, had_mass_err = Array{Array{Float64,2},1}(length(t_sink)), Array{Array{Float64,2},1}(length(t_sink))
had_ff, had_ff_err = Array{Array{Float64,2},1}(length(t_sink)), Array{Array{Float64,2},1}(length(t_sink))
had_vecs, had_vecs_err = Array{Array{Float64,1},1}(length(t_sink)), Array{Array{Float64,1},1}(length(t_sink))
for i in 1:length(t_sink)
    had_mass[i], had_mass_err[i], had_ff[i], had_ff_err[i], had_vecs[i], had_vecs_err[i] = find_mass_ff_and_vecs(had_3ptfn[i], had_2ptfn_src[i], had_2ptfn_snk[i], t_sink[i], t_gen_ev, t_var, t_sn, hide_prog=hide_prog)
end;

In [ ]:
### Fit data ###

if fit_data
    limits, mass_sys, mass_sys_err = scan_range_2ptfn(had_2ptfn_snk[3], t_gen_ev, t_var, t_sn, [8e-9, 0.36], func="cosh", hide_prog=hide_prog)
    mass_stat, mass_stat_err = find_mass_fit(had_2ptfn_snk[3], t_gen_ev, t_var, t_sn, limits, [8e-9, 0.36], func="cosh", hide_prog=hide_prog)
    
    println("\n\nEffective masses extracted from the fit\n\n")
    type_labels = ["Point", "\$G1\$", "\$\\nabla^2 G1\$", "\$G2\$", "\$\\nabla^2 G2\$", "Var", "Var + S/N", "S/N"]
    for x in 1:8
        name = type_labels[x]*" "^(13-length(type_labels[x]))
        @printf "%s: %.4f +- %.4f (stat) +- %.4f (sys)\n" name mass_stat[x] mass_stat_err[x] mass_sys_err[x]
    end
end

### Mass plots

Here are a few mass plots just to verify that things are working correctly.

In [ ]:
type_labels = ["Point", "\$G1\$", "\$\\nabla^2 G1\$", "\$G2\$", "\$\\nabla^2 G2\$", "Var", "Var + S/N", "S/N"]
figure(1,figsize=(16, 16))

for x in 1:8
    subplot(810+x)
    errorbar(0:(length(had_mass[length(t_sink)][x,:])-1), had_mass[length(t_sink)][x,:], yerr=had_mass_err[length(t_sink)][x,:], color="b", ecolor="b", capsize=2, fmt="o")
    text(15., 0.55, type_labels[x], fontsize=20)
    ylabel("\$m_{eff}\$", fontsize=20) 
    xlim(-0.5, size_t/2+0.5)
    ylim(0., 0.7)
    yticks(0.:0.2:0.6, fontsize=16)
    if x == 8
        xlabel("\$\\Delta t\$", fontsize=20)
        xticks(0:5:30, fontsize=16)
    else
        xticks(0:5:30, fontsize=0)
    end
    if fit_data
        times = [(limits[x,1]:limits[x,2])-1;]
        y = [log(cosh_func(i,size_t,[1.,mass_stat[x]])/cosh_func(i+1,size_t,[1.,mass_stat[x]])) for i in limits[x,1]:limits[x,2]]
        yminuserr = [log(cosh_func(i,size_t,[1.,mass_stat[x]-mass_stat_err[x]-mass_sys_err[x]])/cosh_func(i+1,size_t,[1.,mass_stat[x]-mass_stat_err[x]-mass_sys_err[x]])) for i in limits[x,1]:limits[x,2]]
        ypluserr = [log(cosh_func(i,size_t,[1.,mass_stat[x]+mass_stat_err[x]+mass_sys_err[x]])/cosh_func(i+1,size_t,[1.,mass_stat[x]+mass_stat_err[x]+mass_sys_err[x]])) for i in limits[x,1]:limits[x,2]]
        plot(times, y, color="red")
        fill_between(times, yminuserr, ypluserr, alpha=0.3, color="red")
    end
    grid()
end;

And here are the plots plotted together.

In [ ]:
type_labels = ["Point", "\$G1\$", "\$\\nabla^2 G1\$", "\$G2\$", "\$\\nabla^2 G2\$", "Var", "Var + S/N", "S/N"]
colors = ["b", "g", "k", "m", "r"]
markers = ["^", "s", "o", "8", "x", "*"]
figure(2,figsize=(16, 8))

for x in 1:8
    errorbar((0:(length(had_mass[length(t_sink)][x,:])-1))+0.05*(x-1), had_mass[length(t_sink)][x,:], yerr=had_mass_err[length(t_sink)][x,:], color=colors[mod(x-1,length(colors))+1], ecolor=colors[mod(x-1,length(colors))+1], capsize=2, fmt=markers[mod(x-1,length(markers))+1], label=type_labels[x])
    ylabel("\$m_{eff}\$", fontsize=20) 
    xlim(-0.5, size_t/2+0.5)
    ylim(0.2, 0.7)
    yticks(0.2:0.1:0.6, fontsize=16)
    if x == 8
        xlabel("\$\\Delta t\$", fontsize=20)
        xticks(0:5:30, fontsize=16)
    else
        xticks([])
    end
end
legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=8, ncol=4, mode="expand", borderaxespad=0., fontsize=16)
grid();

### Comparison of extracted masses (if fits are available)

In [ ]:
if fit_data
    type_labels = ["Point", "\$G1\$", "\$\\nabla^2 G1\$", "\$G2\$", "\$\\nabla^2 G2\$", "Var", "Var + S/N", "S/N"]
    figure(3, figsize=(16, 8))
    x = 1:8
    y = [mass_stat[i] for i in 1:8]
    yerr = [mass_stat_err[i]+mass_sys_err[i] for i in 1:8]
    errorbar(x, y, yerr=yerr, color="b", ecolor="b", capsize=2, fmt="o")
    xlim(0, 9)
    xticks(x, type_labels, rotation=60, fontsize=16)
    yticks(0.33:0.01:0.39, fontsize=16)
    ylabel("\$m_{eff}\$", fontsize=20)
    grid()
end;

### Charge/form factor plots

In [ ]:
type_labels = ["Point", "\$G1\$", "\$\\nabla^2 G1\$", "\$G2\$", "\$\\nabla^2 G2\$", "Var", "Var + S/N", "S/N"]
colors = ["b", "g", "k", "m", "r"]
markers = ["^", "s", "o", "8", "x", "*"]
figure(4, figsize=(16, 16))
for x in 1:8, y in 1:length(t_sink)
    subplot(810+x)
    errorbar((0:(length(had_ff[y][x,:])-1))+0.1(y-1), had_ff[y][x,:], yerr=had_ff_err[y][x,:], color=colors[y], ecolor=colors[y], capsize=2, fmt=markers[y], label="\$t_{sink}=$(t_sink[y])\$")
    ylabel("\$R\$", fontsize=20)
    text(10., 0.88, type_labels[x], fontsize=20)
    xlim(-0.5, t_sink[length(t_sink)]+0.5)
    ylim(0.4, 1)
    yticks(0.4:0.2:1., fontsize=16)
    if x == 8
        xlabel("\$\\tau\$", fontsize=20)
        xticks(0:5:t_sink[length(t_sink)], fontsize=16)
    else
        xticks(0:5:t_sink[length(t_sink)], fontsize=0)
    end
    if x == 1
        legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=3, ncol=3, mode="expand", borderaxespad=0., fontsize=16)
    end
    grid()
end

In [ ]:
type_labels = ["Point", "\$G1\$", "\$\\nabla^2 G1\$", "\$G2\$", "\$\\nabla^2 G2\$", "Var", "Var + S/N", "S/N"]
colors = ["b", "g", "k", "m", "r"]
markers = ["^", "s", "o", "8", "x", "*"]
fig = figure(figsize=(16, 8))
@manipulate for t_snk in togglebuttons(t_sink, value=t_sink[length(t_sink)], label="\$t_{sink}\$")
    withfig(fig) do
        y = findfirst(t_sink, t_snk)
        for x in 1:8
            errorbar((0:(length(had_ff[y][x,:])-1))+0.05*(x-1), had_ff[y][x,:], yerr=had_ff_err[y][x,:], color=colors[mod(x-1,length(colors))+1], ecolor=colors[mod(x-1,length(colors))+1], capsize=2, fmt=markers[mod(x-1,length(markers))+1], label=type_labels[x])
            ylabel("\$R\$", fontsize=20)
            xlim(-0.5, t_sink[y]+0.5)
            ylim(0.4, 1)
            yticks(0.4:0.2:1., fontsize=16)
            if x == 8
                xlabel("\$\\tau\$", fontsize=20)
                xticks(0:5:t_sink[y], fontsize=16)
            else
                xticks(0:5:t_sink[y], fontsize=0)
            end
        end
        grid()
        legend(bbox_to_anchor=(0., 1.02, 1., .102), loc=8, ncol=4, mode="expand", borderaxespad=0., fontsize=16)
        text(10., 0.88, "\$t_{snk} = $(t_snk)\$", fontsize=20)
    end;
end

### Variational vs S/N vector comparison

In [ ]:
bar_width = 0.25
opacity = 0.4
figure(4, figsize=(12, 6))

bar(1:5, had_vecs[length(t_sink)][1:5], bar_width, alpha=opacity, color="b", ecolor="b", yerr=had_vecs_err[length(t_sink)][1:5], label="Variational")
bar((1:5)+bar_width, had_vecs[length(t_sink)][6:10], bar_width, alpha=opacity, color="r", ecolor="r", yerr=had_vecs_err[length(t_sink)][6:10], label="S/N source")
bar((1:5)+2bar_width, had_vecs[length(t_sink)][11:15], bar_width, alpha=opacity, color="g", ecolor="g", yerr=had_vecs_err[length(t_sink)][11:15], label="S/N sink")

xlabel("Component", fontsize=20)
ylabel("Absolute value", fontsize=20)
xticks((1:5) + 1.5bar_width, ["Point", "\$G1\$", "\$\\nabla^2 G1\$", "\$G2\$", "\$\\nabla^2 G2\$"], fontsize=16)
yticks(0:0.2:1.2, fontsize=16)
ylim(0., 1.2)
legend(fontsize=16)
tight_layout()

In [ ]:
av = mean(had_3ptfn[3],1)[1,1,:,:]

for x in 1:5, y in x:5
    println("$x $y  ",abs((av[x,y]-av[y,x]')/abs(av[x,y])))
end